# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [96]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [97]:
# your code here
from sqlalchemy import create_engine

class Connector:
    def __init__(self, connection_path):
        self.engine = create_engine(connection_path)
        self.conn = self.engine.connect()
    
    def read_sql(self, sql):
        return pd.read_sql(sql, self.conn)

    def to_sql(self, df, table_name, ifexists):
        df.to_sql(table_name, self.conn, if_exists=ifexists)

    def __del__(self):
        print('-----------------------')
        try:
            self.conn.close()
        except:
            print('could not close connection')
        try:
            self.engine.dispose()
        except:
            print('could not close engine')
        finally:
            print('Connector destroyed.')


driver   = 'mysql+pymysql:'
user     = 'root'
password = '53cawentOo69'
ip       = 'localhost'
database = 'orders'
local = Connector(f'{driver}//{user}:{password}@{ip}/{database}')
orders = local.read_sql('SELECT * FROM orders')
orders


,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,397919,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,397920,581587,22899,2011,12,5,12,children s apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,397921,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,397922,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [105]:
# your code here
def get_sales_amount_item(row):
    return row.Quantity * row.UnitPrice

minimal = orders[['Quantity','UnitPrice','CustomerID','Country']]
minimal['TotalPaid'] = minimal.apply(lambda row : get_sales_amount_item(row), axis=1)

paid_by_customer = minimal.groupby(by=['CustomerID','Country'], as_index=False).agg({'TotalPaid':'sum'})
#paid_by_customer = paid_by_customer.sort_values('TotalPaid', ascending=False)
paid_by_customer

/Users/Cunillet/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CustomerID,Country,TotalPaid
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40
...,...,...,...
4342,18280,United Kingdom,180.60
4343,18281,United Kingdom,80.82
4344,18282,United Kingdom,178.05
4345,18283,United Kingdom,2094.88


In [106]:
"""bins = pd.cut(minimal.TotalPaid, 20, labels=list(range(20)))
bins[bins == 19]"""
q = paid_by_customer.quantile([0.95, 0.75], axis=0)
q

,CustomerID,TotalPaid
0.95,17982.8,5774.278
0.75,16776.5,1661.195


In [110]:
def is_add_tag(paid, vip, pref):
    tag = '--'
    if (paid >= float(vip)):
        tag = 'VIP'
    elif (paid >= float(pref)):
        tag = 'Preferred'
    return tag

paid_by_customer['Tag'] = paid_by_customer.apply(lambda row : is_add_tag(row.TotalPaid, q.iloc[0,1], q.iloc[1,1]), axis=1)
paid_by_customer

,CustomerID,Country,TotalPaid,Tag
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,--
...,...,...,...,...
4342,18280,United Kingdom,180.60,--
4343,18281,United Kingdom,80.82,--
4344,18282,United Kingdom,178.05,--
4345,18283,United Kingdom,2094.88,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [120]:
# your code here
vip_country = paid_by_customer[paid_by_customer.Tag == 'VIP'].groupby('Country', as_index=False).count()
vip_country.sort_values('CustomerID', ascending=False)

,Country,CustomerID,TotalPaid,Tag
16,United Kingdom,178,178,178
7,Germany,11,11,11
6,France,9,9,9
15,Switzerland,3,3,3
13,Spain,2,2,2
11,Portugal,2,2,2
8,Japan,2,2,2
4,EIRE,2,2,2
5,Finland,1,1,1
1,Channel Islands,1,1,1


In [121]:
preferred_country = paid_by_customer[paid_by_customer.Tag == 'Preferred'].groupby('Country', as_index=False).count()
preferred_country.sort_values('CustomerID', ascending=False)

,Country,CustomerID,TotalPaid,Tag
24,United Kingdom,756,756,756
10,Germany,28,28,28
9,France,20,20,20
2,Belgium,11,11,11
23,Switzerland,6,6,6
18,Norway,6,6,6
21,Spain,5,5,5
20,Portugal,5,5,5
14,Italy,5,5,5
8,Finland,4,4,4


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [122]:
# your code here
top_country = vip_country + preferred_country
top_country.sort_values('CustomerID', ascending=False)

,Country,CustomerID,TotalPaid,Tag
16,United KingdomLebanon,179.0,179.0,179.0
10,NorwayGermany,29.0,29.0,29.0
9,NetherlandsFrance,21.0,21.0,21.0
7,GermanyEIRE,12.0,12.0,12.0
2,CyprusBelgium,12.0,12.0,12.0
6,FranceDenmark,11.0,11.0,11.0
14,SwedenItaly,6.0,6.0,6.0
8,JapanFinland,6.0,6.0,6.0
4,EIREChannel Islands,5.0,5.0,5.0
15,SwitzerlandJapan,5.0,5.0,5.0
